# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Use statistical modeling and machine learning to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL?p=AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Premium News Earnings Tech Housing Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Premium News Earnings Tech Housing Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

I need to delegate tasks to my team members, gather insights from the Data Analyst, and ensure that the trading strategies align with the user's risk tolerance and preferences.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop trading strategies for AAPL based on insights from Data Analyst and user-defined risk tolerance (Medium) considering trading preferences (Day Trading)", "context": "Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to

Final Answer: Based on the insights gathered from the Yahoo Finance website content for AAPL, it is evident that Apple Inc. (AAPL) is a leading consumer electronics and technology company with a strong market presence. The stock has shown positive returns over various timeframes, including a 6.09% YTD return and a 27.93% return over the past year. With a market cap of 3.58T and a diverse product portfolio ranging from smartphones to wearables, Apple Inc. is well-positioned in the market.

In developing trading strategies for AAPL, considering the current market conditions and the user-defined risk tolerance, it would be essential to analyze the historical performance of the stock, upcoming events such as earnings reports, and potential market trends. Day trading preferences would require a focus on short-term price movements and intraday trading opportunities, leveraging technical analysis and market indicators.

Overall, a strategy that combines a mix of technical analysis, fundamenta

Final Answer: 
After conducting a search on approved trading strategies for AAPL, I found several resources that provide insights into different approaches. Some of the strategies mentioned include using put options, day trading at all-time highs, implementing covered calls for generating additional income, utilizing bullish option plays like call debit spreads, applying simple option buying strategies, and trading Apple CFDs. It is crucial to consider factors such as market conditions, historical performance, upcoming events, and technical/fundamental analysis to maximize profitability and minimize risks when trading AAPL stock. For a comprehensive understanding of these strategies, I recommend reviewing the content from the following sources:

1. Apple Trading Strategies & Insights - LinkedIn
Link: [Apple Trading Strategies & Insights on LinkedIn](https://www.linkedin.com/pulse/apple-trading-strategies-insights-quantifiedstrategies-apmpc)

2. Day Trading Strategy for AAPL All Time Hi

I need to gather a comprehensive risk analysis report for the proposed trading strategies and execution plans for AAPL. I should utilize the expertise of the Risk Advisor to assess and mitigate potential risks effectively.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Risk Advisor",
"task": "Conduct a comprehensive risk analysis report for the proposed trading strategies and execution plans for AAPL. Provide detailed analysis of potential risks and suggest mitigation strategies.",
"context": "After delegating the task of analyzing approved trading strategies for AAPL to the Trading Strategy Developer, I have gathered insights from various sources on different approaches. Some of the strategies mentioned include using put options, day trading at all-time highs, implementing covered calls for generating additional income, utilizing bullish option plays like call debit spreads, applying simple option buying strategies, and trading Apple CFDs. It is essential to conside

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

---

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for AAPL:

Introduction:
Trading strategies for AAPL involve a range of approaches aimed at maximizing profitability while mitigating potential risks. In this analysis, we will assess the risks associated with various proposed trading strategies and provide recommendations for risk mitigation.

1. Put Options Strategy:
- Risk Assessment: Put options provide downside protection but involve the risk of time decay and limited profit potential.
- Mitigation Strategy: Consider using put options in conjunction with other strategies to diversify risk exposure.

2. Day Trading at All-Time Highs:
- Risk Assessment: Day trading at all-time highs can be profitable but carries the risk of sudden price reversals.
- Mitigation Strategy: Implement tight stop-loss orders to limit potential losses and secure profits.

3. Covered Calls Strategy:
- Risk Assessment: Implementing covered calls can generate additional income but may limit upside potential if the stock price rises significantly.
- Mitigation Strategy: Monitor the stock price closely and adjust the covered call strategy accordingly to maximize returns.

4. Bullish Option Plays like Call Debit Spreads:
- Risk Assessment: Bullish option plays like call debit spreads involve the risk of losing the premium paid if the stock price does not move as expected.
- Mitigation Strategy: Consider using call debit spreads in conjunction with technical analysis to identify optimal entry and exit points.

5. Simple Option Buying Strategies:
- Risk Assessment: Simple option buying strategies can be profitable but carry the risk of losing the entire premium paid if the option expires worthless.
- Mitigation Strategy: Diversify option purchases across different strike prices and expiration dates to spread risk.

6. Trading Apple CFDs:
- Risk Assessment: Trading Apple CFDs allows for leveraged trading but involves the risk of margin calls and potential losses exceeding the initial investment.
- Mitigation Strategy: Use appropriate risk management techniques such as setting stop-loss orders and managing leverage effectively.

Conclusion:
In conclusion, the proposed trading strategies for AAPL offer various opportunities for profit but also carry inherent risks that must be carefully managed. By conducting a comprehensive risk analysis and implementing appropriate risk mitigation strategies, traders can better navigate the complexities of trading AAPL stock and achieve their financial objectives.

---

This risk analysis report provides a detailed assessment of the potential risks associated with the proposed trading strategies for AAPL and offers recommendations for mitigating these risks effectively.